## Day 33 Lecture 1 Assignment

In this assignment, we will learn about SVM. We will use the csection dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ds_useful as ds

In [2]:
# Age -  ranges from 18 to 40  
# Delivery number - ranges from 1 to 4
# Delivery time -  {0 = timely , 1 = premature , 2 = latecomer}
# Blood of Pressure - {0 = low , 1 = normal , 2 = high }
# Heart Problem - {0 = apt, 1 = inept }

# Target - Caesarian - {0 = No, 1 = Yes }

csection = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/csection.csv')

In [7]:
csection.head()

,Age,Delivery number,Delivery time,Blood Pressure,Heart Problem,Caesarian
0,22,1,0,2,0,0
1,26,2,0,1,0,1
2,26,2,1,1,0,0
3,28,1,0,2,0,0
4,22,2,0,1,0,1


Recall our c section dataset. Process the data by removing any missing values, creating the appropriate dummy variables and splitting the data into train and test (20% in the test dataset).

In [4]:
# answer below:
ds.missingness_summary(csection, print_log=True)
print('No missing values')

Caesarian          0.0
Heart Problem      0.0
Blood Pressure     0.0
Delivery time      0.0
Delivery number    0.0
Age                0.0
dtype: float64
No missing values


In [36]:
def get_categorical_columns(df, unique_thresh=10, non_binary=False):
    columns = []
    for col in df.columns:
        if df[col].nunique() < unique_thresh:
            if non_binary and df[col].nunique() < 2:
                continue
            columns.append(col)
    return columns

cat_cols = get_categorical_columns(csection, unique_thresh=5)

dum_cols = []
for col in cat_cols:
    if csection[col].nunique() > 2:
        dum_cols.append(col)

csection[dum_cols] = csection[dum_cols].astype('object')

dums = pd.get_dummies(csection.select_dtypes('object'), drop_first=True)

csec_feat = pd.concat([dums, csection.drop(dum_cols, axis=1)], axis=1)
csec_feat.head()

,Delivery number_2,Delivery number_3,Delivery number_4,Delivery time_1,Delivery time_2,Blood Pressure_1,Blood Pressure_2,Age,Heart Problem,Caesarian
0,0,0,0,0,0,0,1,22,0,0
1,1,0,0,0,0,1,0,26,0,1
2,1,0,0,1,0,1,0,26,0,0
3,0,0,0,0,0,0,1,28,0,0
4,1,0,0,0,0,1,0,22,0,1


Generate an SVC model with a linear kernel. Set C=10. Check the score for both train and test. Does the model overfit?

In [13]:
# answer below:
from sklearn.model_selection import train_test_split

X = csection.drop('Caesarian', axis=1)
Y = csection['Caesarian']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=.2, random_state=134)


In [19]:

from sklearn.svm import SVC

svm = SVC(C=10, kernel='linear')
svm.fit(X_train, Y_train)

print('Accuracy on training set', svm.score(X_train, Y_train))
print('Accuracy on testing set', svm.score(X_test, Y_test))

print('There does seem to be overfitting')

Accuracy on training set 0.6875
Accuracy on testing set 0.5
There does seem to be overfitting


If you have determined the model to overfit, pick a value of C based on your conclusion and show that the difference between the scores has improved.

In [28]:
# answer below:

#Since its overfitting, lets increase regularization by lowering C

svm = SVC(C=.01, kernel='linear')
svm.fit(X_train, Y_train)

print('Accuracy on training set', svm.score(X_train, Y_train))
print('Accuracy on testing set', svm.score(X_test, Y_test))

Accuracy on training set 0.5625
Accuracy on testing set 0.625


In [29]:
print('The training score went down, but the testing score increased. They are now closer together.')

The training score went down, but the testing score increased. They are now closer together.


What if we switched up the target variable? Let assume that we know whether a patient received a c section. Let's try to predict whether she has high blood pressure. 

Create the SVC model with a linear kernel with Blood Pressure as the target variable. Report both the train and the test scores.

In [31]:
# answer below:

csection.head()

,Age,Delivery number,Delivery time,Blood Pressure,Heart Problem,Caesarian
0,22,1,0,2,0,0
1,26,2,0,1,0,1
2,26,2,1,1,0,0
3,28,1,0,2,0,0
4,22,2,0,1,0,1


In [45]:
target = 'Blood Pressure'
csection = csection.astype('int64')
dum_cols = get_categorical_columns(csection.drop(target, axis=1), unique_thresh=5, non_binary=True)

csection[dum_cols] = csection[dum_cols].astype('object')

dums = pd.get_dummies(csection[dum_cols], drop_first=True)

csec_feat = pd.concat([dums, csection.drop(dum_cols, axis=1)], axis=1)
csec_feat.head()

,Delivery number_2,Delivery number_3,Delivery number_4,Delivery time_1,Delivery time_2,Heart Problem_1,Caesarian_1,Age,Blood Pressure
0,0,0,0,0,0,0,0,22,2
1,1,0,0,0,0,0,1,26,1
2,1,0,0,1,0,0,0,26,1
3,0,0,0,0,0,0,0,28,2
4,1,0,0,0,0,0,1,22,1


In [46]:
X = csec_feat.drop('Blood Pressure', axis=1)
Y= csec_feat['Blood Pressure']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=134)

In [50]:
svm = SVC(C=.1, kernel='linear', decision_function_shape='ovr')
svm.fit(X_train, Y_train)

print('Accuracy on training set', svm.score(X_train, Y_train))
print('Accuracy on testing set', svm.score(X_test, Y_test))

Accuracy on training set 0.5
Accuracy on testing set 0.5


Now try to generate the model using the one vs. one decision function.

In [51]:
# answer below:
svm = SVC(C=.1, kernel='linear', decision_function_shape='ovo')
svm.fit(X_train, Y_train)

print('Accuracy on training set', svm.score(X_train, Y_train))
print('Accuracy on testing set', svm.score(X_test, Y_test))

Accuracy on training set 0.5
Accuracy on testing set 0.5
